In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.preprocessing import LabelEncoder
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sklearn.model_selection import GridSearchCV
import optuna
from sklearn.metrics import accuracy_score, mean_absolute_error
#libs we need

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
import seaborn as sns
train_df = train_df.drop(['Id', 'SaleType', 'SaleCondition'], axis = 1)
select_obj = train_df.select_dtypes('object').columns
for i in select_obj:
    train_df[i] = LabelEncoder().fit_transform(train_df[i])
    test_df[i] = LabelEncoder().fit_transform(test_df[i])
for x in train_df.columns:
    if train_df[x].isnull().sum() > 500:train_df = train_df.drop(x, axis = 1)
    else:
        train_df[x] = train_df[x].fillna(train_df[x].median())
        if x!= "SalePrice": test_df.fillna(test_df[x].median())

In [3]:
X, y = train_df.drop('SalePrice', axis = 1), (train_df['SalePrice'])

In [4]:
def objective(trial):
    # Определение параметров, подлежащих оптимизации
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
    train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 2)
    model.fit(train_X, train_y)
    y_pred = model.predict(val_X)
    accuracy = mean_absolute_error(val_y, y_pred)
    return accuracy

In [5]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials = 100)

[I 2023-11-19 18:39:28,516] A new study created in memory with name: no-name-7d7937c5-61f9-4800-b5ab-512c531a9704
[I 2023-11-19 18:39:28,826] Trial 0 finished with value: 21689.769667108347 and parameters: {'n_estimators': 27, 'max_depth': 7, 'min_samples_split': 10}. Best is trial 0 with value: 21689.769667108347.
[I 2023-11-19 18:39:29,431] Trial 1 finished with value: 22301.1279169329 and parameters: {'n_estimators': 57, 'max_depth': 6, 'min_samples_split': 8}. Best is trial 0 with value: 21689.769667108347.
[I 2023-11-19 18:39:30,171] Trial 2 finished with value: 21154.280545201436 and parameters: {'n_estimators': 64, 'max_depth': 7, 'min_samples_split': 2}. Best is trial 2 with value: 21154.280545201436.
[I 2023-11-19 18:39:31,175] Trial 3 finished with value: 21281.158074941824 and parameters: {'n_estimators': 89, 'max_depth': 7, 'min_samples_split': 2}. Best is trial 2 with value: 21154.280545201436.
[I 2023-11-19 18:39:31,300] Trial 4 finished with value: 32335.882133419425 and

In [6]:
print(study.best_params)
print(study.best_value)

{'n_estimators': 96, 'max_depth': 10, 'min_samples_split': 5}
20165.338883568184


In [7]:
final_model = RandomForestRegressor(n_estimators= 71, max_depth = 10, min_samples_split = 6)
features = test_df.drop(['Id', 'SaleType', 'SaleCondition'], axis = 1).columns
final_model.fit(X, y)
for x in test_df[features]:
    if test_df[x].isnull().sum() > 500:test_df = test_df.drop(x, axis = 1)
    else:
        test_df[x] = test_df[x].fillna(test_df[x].median())
X = test_df[features].ffill()


In [8]:
predictions = final_model.predict(X)

In [9]:
submission = pd.DataFrame()
submission['Id'] = test_df['Id']
submission['SalePrice'] = predictions
submission.to_csv('submision.csv', index = False)

In [10]:
#That is my first model of ML so don't rate it like a pro work.. 